# IMPC_APIを用いてPhenodiデータを取得する

# セットアップ

In [1]:
# Move up to top directory
import os
from pathlib import Path

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir('../')

print(os.getcwd())

/mnt/e/Research/TSUMUGI-dev-main/notebooks/notebooks-experiments
/mnt/e/Research/TSUMUGI-dev-main


In [2]:
from pathlib import Path
from pprint import pprint
from collections import defaultdict, Counter
from itertools import combinations
import csv
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

P = print
PP = pprint
C = Counter


# 実験

In [3]:
from impc_api import solr_request, batch_solr_request


In [7]:
num_found, df = solr_request(
    core="genotype-phenotype",
    params={
         "q": "*:*",
         "rows": 0,
         "facet": "on",
         "facet.field": "zygosity",
         "facet.limit": 15,
         "facet.mincount": 1,
    }
)
df


Your request:
https://www.ebi.ac.uk/mi/impc/solr/genotype-phenotype/select?q=%2A%3A%2A&rows=0&facet=on&facet.field=zygosity&facet.limit=15&facet.mincount=1

Number of found documents: 69489



,zygosity,count_per_category
0,homozygote,54170
1,heterozygote,14551
2,hemizygote,768


,zygosity,count_per_category
0,homozygote,54170
1,heterozygote,14551
2,hemizygote,768


In [8]:
num_found, df = solr_request(
    core='phenodigm', 
    params={
        'q': '*:*',
        'rows': 10, 
    }
)


Your request:
https://www.ebi.ac.uk/mi/impc/solr/phenodigm/select?q=%2A%3A%2A&rows=10

Number of found documents: 10152848



,type,gene_id,hgnc_gene_id
0,gene_gene,MGI:102498,HGNC:7459
1,gene_gene,MGI:102500,HGNC:7456
2,gene_gene,MGI:99927,HGNC:7414
3,gene_gene,MGI:102501,HGNC:7427
4,gene_gene,MGI:102496,HGNC:7461
5,gene_gene,MGI:102497,HGNC:7460
6,gene_gene,MGI:102502,HGNC:7422
7,gene_gene,MGI:102499,HGNC:7458
8,gene_gene,MGI:102495,HGNC:7462
9,gene_gene,MGI:102504,HGNC:7419


In [17]:
# Overlapeed phenotypeの中で、maleとfemaleを含むものを抽出する
overlapped_phenotype = df_similarity_with_sexual_dimorphism['Overlapped phenotype'].tolist()

import ast

# 各文字列をパースしてリストに変換
parsed_lists = [ast.literal_eval(item) for item in overlapped_phenotype]

# 全てのリストを結合
flattened_list = []
for sublist in parsed_lists:
    flattened_list.extend(sublist)

P(flattened_list[:5])

['decreased bone mineral content (Homo, Female)', 'decreased bone mineral density (Homo)', 'decreased bone mineral content (Homo, Female)', 'decreased bone mineral content (Homo, Female)', 'decreased bone mineral content (Homo, Female)']


In [18]:

phenotype_with_sexual_dimorphism = [p for p in flattened_list if "Male" in p or "Female" in p]

P(phenotype_with_sexual_dimorphism[:5])

['decreased bone mineral content (Homo, Female)', 'decreased bone mineral content (Homo, Female)', 'decreased bone mineral content (Homo, Female)', 'decreased bone mineral content (Homo, Female)', 'decreased bone mineral content (Homo, Female)']


In [20]:
Counter(phenotype_with_sexual_dimorphism).most_common(100)

[('hyperactivity (Homo, Female)', 13781),
 ('increased circulating alkaline phosphatase level (Homo, Female)', 3981),
 ('decreased bone mineral content (Homo, Female)', 3132),
 ('decreased grip strength (Homo, Female)', 2839),
 ('increased grip strength (Homo, Female)', 1835),
 ('decreased locomotor activity (Homo, Female)', 1821),
 ('hyperactivity (Hetero, Female)', 1246),
 ('decreased bone mineral density (Homo, Female)', 1208),
 ('increased total body fat amount (Homo, Female)', 917),
 ('abnormal bone structure (Homo, Female)', 809),
 ('increased fasting circulating glucose level (Homo, Female)', 682),
 ('increased circulating alkaline phosphatase level (Hetero, Female)', 612),
 ('increased bone mineral content (Homo, Female)', 572),
 ('decreased lean body mass (Homo, Female)', 525),
 ('increased grip strength (Hetero, Female)', 509),
 ('decreased grip strength (Hetero, Female)', 434),
 ('decreased exploration in new environment (Homo, Female)', 292),
 ('decreased body length (Homo,